# Лабораторная работа 4. Вариант 3. Группа 343

## Задание
Найти решение задачи

$\frac{\partial u}{\partial t}=Lu+f(x,t),0<x<1,0<1\leq0.1,$

$u(x,0)=\varphi(x),0\leq x\leq1$

$(\alpha_1(t)u-\alpha_2(t)\frac{\partial u}{\partial x})|_{x=0}=\alpha(t), 0\leq t\leq0.1,$

$(\beta_1(t)u+\beta_2(t)\frac{\partial u}{\partial x})|_{x=1}=\beta(t), 0\leq t\leq0.1,$

используя различные разностные схемы:

+ явную схему порядка $O(h^2+\tau)$ с аппроксимацией производных в граничных условиях с порядком $O(h^2)$;
+ схему с весами при $\sigma=0,\sigma=1,\sigma=1/2$ с аппроксимацией производных в граничных условиях с порядком $O(h)$.

### Условие
$\frac{\partial u}{\partial t}=\frac{\partial^2 u}{\partial {x^2}}+\frac{\partial u}{\partial x}+f(x,t),$

$u(x,0)=\varphi(x),0\leq x\leq 1,$

$u(0,t)=\alpha(t), \frac{\partial u}{\partial x}|_{x=1}=\beta(t),0\leq t\leq 0.1.$

### Функции для тестирования
$u_1(x, t) = x^3 + t^3$

$u_2(x, t) = x^3t^3$

$u_3(x, t) = cos(2x) + sin(2t + 1)$

$u_4(x, t) = cos(2x)sin(2t + 1)$

$u_5(x, t) = 2x - t$

In [1]:
import numpy as np
from numpy import linalg as LA
from tabulate import tabulate

N = 5
M = 5
X_RANGE = (0, 1)
T_RANGE = (0, 0.1)
STEP_X = X_RANGE[1] / N
STEP_T = T_RANGE[1] / M

ALPHA_1 = 1
ALPHA_2 = 0
BETA_1 = 0
BETA_2 = 1

In [2]:
# (function, first derivative with respect to x, second derivative with respect to x, first derivative with respect to t)
def u1(x, t):
    u = x ** 3 + t ** 3
    dxu = 3 * x ** 2
    d2xu = 6 * x
    dtu = 3 * t ** 2
    return (u, dxu, d2xu, dtu)


def u2(x, t):
    u = x ** 3 * t ** 3
    dxu = 3 * x ** 2 * t ** 3
    d2xu = 6 * x * t ** 3
    dtu = 3 * x ** 3 * t ** 2
    return (u, dxu, d2xu, dtu)


def u3(x, t):
    u = np.cos(2 * x) + np.sin(2 * t + 1)
    dxu = -2 * np.sin(2 * x)
    d2xu = -4 * np.cos(2 * x)
    dtu = 2 * np.sin(2 * t + 1)
    return (u, dxu, d2xu, dtu)


def u4(x, t):
    u = np.cos(2 * x) * np.sin(2 * t + 1)
    dxu = -2 * np.sin(2 * x) * np.sin(2 * t + 1)
    d2xu = -4 * np.cos(2 * x) * np.sin(2 * t + 1)
    dtu = 2 * np.cos(2 * t + 1) * np.cos(2 * x)
    return (u, dxu, d2xu, dtu)

def u5(x, t):
    u = 2 * x - t
    dxu = 2
    d2xu = 0
    dtu = -1
    return (u, dxu, d2xu, dtu)


US = [u1, u2, u3, u4, u5]

In [3]:
def a(x, t):
    return 1


def b(x, t):
    return 1


def c(x, t):
    return 0

In [4]:
def get_accurate_solution(u, x, t):
    solution = u(x, t)[0]
    f_value = u(x, t)[3] - u(x, t)[2] - u(x, t)[1]
    return (solution, f_value)

$u_0^k(\alpha_1(t_k)+\frac{3}{2h}\alpha_2(t_k))-\alpha_2(t_k)\frac{4u_1^k-u_2^k}{2h}=\alpha(t_k)$

$u_0^k=\frac{\alpha_2(t_k)\frac{4u_1^k-u_2^k}{2h}+\alpha(t_k)}{\alpha_1(t_k)+\frac{3}{2h}\alpha_2(t_k)}$

$u_0^k=\frac{\alpha_2(t_k)(4u_1^k-u_2^k)+2h\alpha(t_k)}{2h\alpha_1(t_k)+3\alpha_2(t_k)}$

-------------

$u_N^k(\beta_1(t_k)+\frac{3}{2h}\beta_2(t_k))-\beta_2(t_k)\frac{4u_{N-1}^k - u_{N-2}^k}{2h}=\beta(t_k)$

$u_N^k=\frac{\beta_2(t_k)\frac{4u_{N-1}^k - u_{N-2}^k}{2h}+\beta(t_k)}
{\beta_1(t_k)+\frac{3}{2h}\beta_2(t_k)}$

$u_N^k=\frac{\beta_2(t_k)(4u_{N-1}^k - u_{N-2}^k)+2h\beta(t_k)}{2h\beta_1(t_k)+3\beta_2(t_k)}$

-------------

$L_hu_i^k=a(x_i, t_k)\frac{u_{i+1}^k-2u_i^k+u_{i-1}^k}{h^2}+b(x_i,t_k)\frac{u_{i+1}^k-u_{i-1}^k}{2h}+c(x_i, t_k)u_i^k$

In [5]:
def get_solution_grid(u, a, b, c):
    # us[i, k] <-> (u_i)^k
    us = np.zeros((N + 1, M + 1))
    # u: (x, t) -> (u, dxu, d2xu, dtu)
    def put_u_i_k(i, k):
        # find (u_i)^0
        x_i = X_RANGE[0] + i * STEP_X
        t_k1 = T_RANGE[0] + (k - 1) * STEP_T
        t_k = t_k1 + STEP_T
        
        if k == 0:
            us[i, k] = u(x_i, T_RANGE[0])[0]
            return
        
        # find (u_0)^k
        if i == 0:
            alpha = ALPHA_1 * u(0, t_k)[0] + ALPHA_2 * u(0, t_k)[1]
            u0k =  ALPHA_2 * (4 * us[1, k] - us[2, k]) + 2 * STEP_X * alpha
            u0k /= 2 * STEP_X * ALPHA_1 + 3 * ALPHA_2
            us[i, k] = u0k
            return
        
        #find (u_N)^k
        if i == N:
            beta = BETA_1 * u(1, t_k)[0] + BETA_2 * u(1, t_k)[1]
            uNk = BETA_2 * (4 * us[N - 1, k] - us[N - 2, k]) + 2 * STEP_X * beta
            uNk /=2 * STEP_X * BETA_1 + 3 * BETA_2
            us[i, k] = uNk
            return
    
        # (u_i)^k-1
        u_prev = us[i, k - 1]
        # find (u_i)^k
        res = u_prev
        
        function_val = get_accurate_solution(u, x_i, t_k1)[1]
        # find Lh
        lh = \
            a(x_i, t_k) * (us[i + 1, k] - 2 * us[i, k] + us[i - 1, k]) / STEP_X ** 2 + \
            0.5 * b(x_i, t_k) * (us[i + 1, k] - us[i - 1, k]) / STEP_X + \
            c(x_i, t_k) * us[i, k]
        
        us[i, k] = u_prev + STEP_T * (lh * u_prev + function_val)
        
    for i in range(0, N + 1):
        put_u_i_k(i, 0)
        
    for k in range(1, M + 1):
        for i in range(1, N):
            put_u_i_k(i, k)
        put_u_i_k(0, k)
        put_u_i_k(N, k)
            
    return us

In [6]:
def get_solution_grid_weights(u, a, b, c):
    # us[i, k] <-> (u_i)^k
    us = np.zeros((N + 1, M + 1))
    return us

In [7]:
def print_result(u, is_explicit):
    if is_explicit:
        print('Явный результат:')
        us = get_solution_grid(u, a, b, c)
    else:
        print('Неявный результат результат:')
        us = get_solution_grid_weights(u, a, b, c)
        
    us_headers = ['t \ x']
    for i in range(0, N + 1):
        us_headers.append(str(round(X_RANGE[0] + i * STEP_X, 2)))
        
    us_table = []
    for k in range(0, M + 1):
        us_table.append([str(round(T_RANGE[0] + k * STEP_T, 2))])
        
    for i in range(0, N + 1):
        for k in range(0, M + 1):
            us_table[k].append(str(us[i, k]))            
        
    print(tabulate(us_table, headers=us_headers))
    
    print('Точность:')
    if is_explicit:
        acc_headers =  ['h', 'tau', '||J_ex - u^(h, tau)||', '||u^(h, tau) - u^(2h, tau1)||']
    else:
        acc_headers =  ['h', 'tau', '||J_ex - u^(h, tau)||', '||u^(h, tau) - u^(2h, tau)||']

In [8]:
for i in range(0, len(US)):
    print(f'Результат для u_{i + 1}\n')
    print_result(US[i], True)
    print()
    print_result(US[i], False)
    print('\n\n\n\n')

Результат для u_1

Явный результат:
  t | x       0.0        0.2          0.4         0.6          0.8       1.0
-------  --------  ---------  -----------  ----------  -----------  --------
   0     0          0.008      0.064        0.216       0.512       1
   0.02  8e-06     -0.0184     0.00587008   0.122971    0.405932    0.900253
   0.04  6.4e-05   -0.044776  -0.0518242    0.0265268   0.276402    0.759694
   0.06  0.000216  -0.07108   -0.107671    -0.0682625   0.133608    0.600898
   0.08  0.000512  -0.097264  -0.160342    -0.156721   -0.00999909  0.438908
   0.1   0.001     -0.12328   -0.208663    -0.235221   -0.142957    0.287798

Неявный результат результат:
  t | x    0.0    0.2    0.4    0.6    0.8    1.0
-------  -----  -----  -----  -----  -----  -----
   0         0      0      0      0      0      0
   0.02      0      0      0      0      0      0
   0.04      0      0      0      0      0      0
   0.06      0      0      0      0      0      0
   0.08      0      0    